In [23]:
import pandas as pd
import seaborn as sns
from collections import OrderedDict

In [24]:
df = pd.read_csv('data.csv')

In [25]:
df.head(2)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


In [33]:
df= df.drop(columns=["sex","children","smoker","region"],axis=1)

In [34]:
def custom_summary(df):
    result =[]
    for col in list(df.columns):
        stats = OrderedDict({"featureName":col,
                            "count":df[col].count(),
                            "datatype":df[col].dtype,
                            "min":df[col].min(),
                            "Q1":df[col].quantile(0.25),
                            "Q2":df[col].quantile(0.5),
                            "Q3":df[col].quantile(0.75),
                            "Q4":df[col].quantile(1.0),
                            "mean":df[col].mean(),
                            "stdv":df[col].std(),
                            "var":df[col].var(),
                            "kurt":df[col].kurt(),
                            "skew":df[col].skew(),
                            "range":df[col].min() - df[col].max(),
                            "IQR": df[col].quantile(0.75) - df[col].quantile(0.25)
                            })
        result.append(stats)
    resultdf = pd.DataFrame(data=result)
    return resultdf
    

In [35]:
custom_summary(df)

,featureName,count,datatype,min,Q1,Q2,Q3,Q4,mean,stdv,var,kurt,skew,range,IQR
0,age,1338,int64,18.0000,27.00000,39.000,51.000000,64.00000,39.207025,14.049960,1.974014e+02,-1.245088,0.055673,-46.00000,24.000000
1,bmi,1338,float64,15.9600,26.29625,30.400,34.693750,53.13000,30.663397,6.098187,3.718788e+01,-0.050732,0.284047,-37.17000,8.397500
2,charges,1338,float64,1121.8739,4740.28715,9382.033,16639.912515,63770.42801,13270.422265,12110.011237,1.466524e+08,1.606299,1.515880,-62648.55411,11899.625365
